In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from IPython.display import FileLink
import re

In [3]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):
    price = ""

    # Try different price locations
    for price_id in ['priceblock_ourprice', 'priceblock_dealprice']:
        price_tag = soup.find("span", attrs={'id': price_id})
        if price_tag:
            price = price_tag.get_text(strip=True)
            break

    # If price is still empty, try new Amazon format
    if not price:
        whole = soup.find("span", attrs={'class': 'a-price-whole'})
        fraction = soup.find("span", attrs={'class': 'a-price-fraction'})
        if whole:
            price = f"${whole.get_text(strip=True).replace('.', '')}"
            if fraction:
                price += f".{fraction.get_text(strip=True)}"

    return price

# Function to extract Product Rating
def get_rating(soup):
    rating = ""
    
    rating_tag = soup.find("span", attrs={'class': 'a-icon-alt'})
    if rating_tag:
        rating_match = re.search(r"(\d+(\.\d+)?)", rating_tag.get_text(strip=True))
        if rating_match:
            rating = rating_match.group(1)  # Extract only the numeric rating
    
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    review_count = ""
    
    review_tag = soup.find("span", attrs={'id': 'acrCustomerReviewText'})
    if review_tag:
        review_match = re.search(r"([\d,]+)", review_tag.get_text(strip=True))
        if review_match:
            review_count = review_match.group(1)  # Extract only the numeric count
    
    return review_count


# Function to extract Availability Status
def get_availability(soup):
    available = "Not Available"
    
    availability_tag = soup.find("div", attrs={'id': 'availability'})
    if availability_tag:
        availability_text = availability_tag.get_text(strip=True)
        stock_match = re.search(r"(\d+)\s+left in stock", availability_text, re.IGNORECASE)
        if stock_match:
            available = f"{stock_match.group(1)} left in stock"

    return available


In [5]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+5&crid=1SEHXMKWYXAIV&sprefix=pla%2Caps%2C110&ref=nb_sb_ss_ts-doa-p_ci_hl-bn-left_1_3"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


/var/folders/y4/0kw5_lhd6cv7bj8lfy2ztm200000gn/T/ipykernel_40781/729700019.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [7]:
amazon_df

,title,price,rating,reviews,availability
1,PlayStation®5 console (slim),$499.00,4.6,"5,413",Not Available
2,PlayStation 5 Pro Console,$699.00,4.3,733,Not Available
3,PlayStation PS5 Console - Fortnite Cobalt Star...,$459.99,4.5,255,Not Available
4,PlayStation®5 console (slim) with EA SPORTS Co...,$528.97,4.9,52,Not Available
5,PlayStation 5 Digital Edition (Renewed),$409.99,3.4,136,9 left in stock
6,Sony PlayStation 5 Console (Renewed),$424.95,3.8,548,Not Available
7,PlayStation 5 Console (Renewed),$439.97,3.8,79,Not Available
8,PlayStation PS5 Console – God of War Ragnarök ...,$569.99,4.8,"13,605",Not Available
9,Sony PlayStation 5 Disc Version (Renewed),$439.99,3.7,128,Not Available
10,PlayStation 5 Console - Marvel’s Spider-Man 2 ...,$569.99,4.8,"4,687",Not Available


In [11]:
# Save DataFrame as CSV
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

# Create a download link
FileLink("amazon_data.csv")

/Users/hemangi/Documents/Python Projects/amazon_data.csv